# Database Setup and Connection Test

This notebook tests the connection to the PostgreSQL database and sets up the necessary tables (`accounts` and `transactions`) for the SimpleFIN data.

In [ ]:
import os
import psycopg2

def setup_database():
    print(f"Current Working Directory: {os.getcwd()}")
    
    # Try to find .env in common locations
    possible_paths = [
        '.env',
        os.path.join(os.getcwd(), '.env'),
        '/home/danielaroko/applications/data_exploration/simplefin/.env'
    ]
    
    env_path = None
    for path in possible_paths:
        if os.path.exists(path):
            env_path = path
            print(f"Found .env at: {env_path}")
            break
    
    if not env_path:
        print("Error: .env file not found in searched locations.")
        return

    # Load .env variables manually
    env_vars = {}
    try:
        with open(env_path, 'r') as f:
            for line in f:
                if '=' in line:
                    key, value = line.strip().split('=', 1)
                    env_vars[key] = value
    except Exception as e:
        print(f"Error reading .env file: {e}")
        return

    print(f"Connecting with: User={env_vars.get('DB_USER')}, DB={env_vars.get('DB_NAME')}")

    try:
        conn = psycopg2.connect(
            host=env_vars.get('DB_HOST', 'localhost'),
            database=env_vars.get('DB_NAME'),
            user=env_vars.get('DB_USER'),
            password=env_vars.get('DB_PASS')
        )
        print("Successfully connected to the database!")
        
        cur = conn.cursor()
        
        # 1. Create ACCOUNTS table
        print("Checking/Creating 'accounts' table...")
        cur.execute("""
            CREATE TABLE IF NOT EXISTS accounts (
                account_id VARCHAR(255) PRIMARY KEY,
                name VARCHAR(255),
                currency VARCHAR(10),
                balance NUMERIC(15, 2),
                balance_date TIMESTAMP,
                last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        """)
        
        # 2. Create TRANSACTIONS table
        print("Checking/Creating 'transactions' table...")
        cur.execute("""
            CREATE TABLE IF NOT EXISTS transactions (
                transaction_id VARCHAR(255) PRIMARY KEY,
                account_id VARCHAR(255) REFERENCES accounts(account_id),
                posted TIMESTAMP,
                amount NUMERIC(15, 2),
                description TEXT
            );
        """)
        
        # 3. Add Category Column if needed
        print("Checking for 'category' column in 'transactions'...")
        cur.execute("""
            DO $$
            BEGIN
                IF NOT EXISTS (SELECT 1 FROM information_schema.columns 
                               WHERE table_name='transactions' AND column_name='category') THEN
                    ALTER TABLE transactions ADD COLUMN category VARCHAR(100);
                    RAISE NOTICE 'Added category column';
                END IF;
            END
            $$;
        """)
        
        conn.commit()
        print("Tables are ready.")
        
        # Verify
        cur.execute("SELECT count(*) FROM accounts;")
        acct_count = cur.fetchone()[0]
        cur.execute("SELECT count(*) FROM transactions;")
        tx_count = cur.fetchone()[0]
        
        print(f"Current DB Status: {acct_count} accounts, {tx_count} transactions.")
        
        cur.close()
        conn.close()

    except Exception as e:
        print(f"Database Error: {e}")

setup_database()